# Implement models with PyTorch to allow for GPU cuda compatability

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [16]:
def train_model(input_name:str, output_name:str) -> None:
    # load data
    df = pd.read_csv(input_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to numpy
    X = np.array(X)
    y = np.array(y)

    # split and scale the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # convert data to cuda tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).cuda()
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).cuda()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).cuda()
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).cuda()

    # create batches for the data to reduce noise
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    # define model (should probably use command line arguments in the future)
    class ANN_Model(nn.Module):
        def __init__(self, input_size):
            super(ANN_Model, self).__init__()
            self.fc1 = nn.Linear(input_size, 512)
            self.fc2 = nn.Linear(512, 512)
            self.fc3 = nn.Linear(512, 1)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.1)
            
        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.relu(self.fc2(x))
            x = self.dropout(x)
            x = self.fc3(x)
            return x

    # create an instance of the model
    model = ANN_Model(input_size=X_train.shape[1]).cuda()
    device = torch.device("cuda") # replace with "cpu" if this gives error
    model.to(device)

    # define the loss function and optimizer
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    # tain the model
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        for batch_inputs, batch_targets in train_loader:
            # move batch to gpu
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            
            # forward pass
            outputs = model(batch_inputs)
            loss = loss_function(outputs, batch_targets.unsqueeze(1))
            
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # print status
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

    # print accuracy
    model.eval()
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    outputs = model(X_test_tensor)
    predicted_labels = torch.round(torch.sigmoid(outputs))
    accuracy = (predicted_labels == y_test_tensor.unsqueeze(1)).sum().item() / len(y_test_tensor)
    print(f"Accuracy: {accuracy}")

    torch.save(model.state_dict(), output_name)

In [17]:
train_model("csvs/ml_X.csv", "built_models/torch_ml.pth")

Epoch 1/50, Loss: 0.6841520071029663
Epoch 2/50, Loss: 0.5988947749137878
Epoch 3/50, Loss: 0.701819896697998
Epoch 4/50, Loss: 0.5737534761428833
Epoch 5/50, Loss: 0.6386070251464844
Epoch 6/50, Loss: 0.5079917311668396
Epoch 7/50, Loss: 0.5501521825790405
Epoch 8/50, Loss: 0.3771301805973053
Epoch 9/50, Loss: 0.4997420310974121
Epoch 10/50, Loss: 0.39194074273109436
Epoch 11/50, Loss: 0.3524281680583954
Epoch 12/50, Loss: 0.3537384569644928
Epoch 13/50, Loss: 0.3414645195007324
Epoch 14/50, Loss: 0.2766386866569519
Epoch 15/50, Loss: 0.26858076453208923
Epoch 16/50, Loss: 0.3695208728313446
Epoch 17/50, Loss: 0.44009679555892944
Epoch 18/50, Loss: 0.2602851986885071
Epoch 19/50, Loss: 0.35198238492012024
Epoch 20/50, Loss: 0.05918464437127113
Epoch 21/50, Loss: 0.16847197711467743
Epoch 22/50, Loss: 0.15289953351020813
Epoch 23/50, Loss: 0.16174697875976562
Epoch 24/50, Loss: 0.1696508228778839
Epoch 25/50, Loss: 0.23475909233093262
Epoch 26/50, Loss: 0.2084735482931137
Epoch 27/50, 

In [8]:
train_model("csvs/ou_X.csv", "built_models/torch_ou.pth")

Epoch 1/35, Loss: 0.6859707236289978
Epoch 2/35, Loss: 0.6633058786392212
Epoch 3/35, Loss: 0.7163214087486267
Epoch 4/35, Loss: 0.6774387359619141
Epoch 5/35, Loss: 0.704948902130127
Epoch 6/35, Loss: 0.6870636343955994
Epoch 7/35, Loss: 0.7001285552978516
Epoch 8/35, Loss: 0.7126855254173279
Epoch 9/35, Loss: 0.6875893473625183
Epoch 10/35, Loss: 0.6950677633285522
Epoch 11/35, Loss: 0.6983870267868042
Epoch 12/35, Loss: 0.6973642706871033
Epoch 13/35, Loss: 0.7161115407943726
Epoch 14/35, Loss: 0.6760515570640564
Epoch 15/35, Loss: 0.6917952299118042
Epoch 16/35, Loss: 0.7015635371208191
Epoch 17/35, Loss: 0.6770138740539551
Epoch 18/35, Loss: 0.6813313364982605
Epoch 19/35, Loss: 0.6985925436019897
Epoch 20/35, Loss: 0.7044984698295593
Epoch 21/35, Loss: 0.6880567073822021
Epoch 22/35, Loss: 0.7080903053283691
Epoch 23/35, Loss: 0.6672552824020386
Epoch 24/35, Loss: 0.7077500820159912
Epoch 25/35, Loss: 0.6926031112670898
Epoch 26/35, Loss: 0.686112105846405
Epoch 27/35, Loss: 0.68

In [9]:
train_model("csvs/pl_X.csv", "built_models/torch_pl.pth")

Epoch 1/35, Loss: 0.6320258378982544
Epoch 2/35, Loss: 0.7509336471557617
Epoch 3/35, Loss: 0.7372971773147583
Epoch 4/35, Loss: 0.6230863332748413
Epoch 5/35, Loss: 0.5309690237045288
Epoch 6/35, Loss: 0.7798155546188354
Epoch 7/35, Loss: 0.6450618505477905
Epoch 8/35, Loss: 0.5251476168632507
Epoch 9/35, Loss: 0.6102474331855774
Epoch 10/35, Loss: 0.543899416923523
Epoch 11/35, Loss: 0.5781813263893127
Epoch 12/35, Loss: 0.3832085132598877
Epoch 13/35, Loss: 0.5380699634552002
Epoch 14/35, Loss: 0.5121578574180603
Epoch 15/35, Loss: 0.4474504888057709
Epoch 16/35, Loss: 0.6427471041679382
Epoch 17/35, Loss: 0.3711192011833191
Epoch 18/35, Loss: 0.32513242959976196
Epoch 19/35, Loss: 0.5785415172576904
Epoch 20/35, Loss: 0.17052845656871796
Epoch 21/35, Loss: 0.6557630896568298
Epoch 22/35, Loss: 0.3618062436580658
Epoch 23/35, Loss: 0.160598024725914
Epoch 24/35, Loss: 0.17308932542800903
Epoch 25/35, Loss: 0.07411102205514908
Epoch 26/35, Loss: 0.6442384719848633
Epoch 27/35, Loss: 